# 第6章: 機械学習
本章では，Fabio Gasparetti氏が公開しているNews Aggregator Data Setを用い，ニュース記事の見出しを「ビジネス」「科学技術」「エンターテイメント」「健康」のカテゴリに分類するタスク（カテゴリ分類）に取り組む．

# 50. データの入手・整形
News Aggregator Data Setをダウンロードし、以下の要領で学習データ（train.txt），検証データ（valid.txt），評価データ（test.txt）を作成せよ．

1. ダウンロードしたzipファイルを解凍し，readme.txtの説明を読む．
2. 情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．
3. 抽出された事例をランダムに並び替える．
4. 抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれtrain.txt，valid.txt，test.txtというファイル名で保存する．ファイルには，１行に１事例を書き出すこととし，カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）．

学習データと評価データを作成したら，各カテゴリの事例数を確認せよ．

In [83]:
!ls files/NewsAggregatorDataset/

2pageSessions.csv check.txt         newsCorpora.csv   readme.txt


In [85]:
!head -210713 files files/NewsAggregatorDataset/newsCorpora.csv | tail -1

head: Error reading files
210713	"The Best Reactions To The Supposed Video of Solange Knowles & Jay Z  ...	http://www.hiphopdx.com/index/news/id.28728/title.-the-best-reactions-to-the-supposed-video-of-solange-knowles-jay-z-fighting-in-an-elevator-list-by-time/	HipHopDX	e	dku0uRoeehpC9JM1RoZ4n0fg8cyoM	www.hiphopdx.com	1399983366398


"The Best Reactions To The Supposed Video of Solange Knowles & Jay Z  ...
## ↑ダブルクオーテーションが片側しかないとDictreaderがうまく読み込まないので修正する

In [87]:
import csv


PUBLISHERS = ('Reuters','Huffington Post','Businessweek','Contactmusic.com','Daily Mail')
CATEGORIES = ('b','t','e','m')

y_X_list = []

with open('files/NewsAggregatorDataset/newsCorpora.csv') as f:
    reader = csv.DictReader(f,delimiter='\t',fieldnames=["id","title","url","publisher","category","story","hostname","timestamp"])
    y_X_list = [(row['category'],row['title']) for row in reader if row['publisher'] in PUBLISHERS and row['category'] in CATEGORIES]


In [88]:
from sklearn.model_selection import train_test_split
from typing import List
import csv
y = [y for y, x in  y_X_list]
X = [x for y, x in  y_X_list]


print(f"y len :{len(y)}, X len: {len(X)}")

X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.20, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_tmp, y_tmp, test_size=0.50, random_state=42)

print(f"train:{len(X_train)}, valid:{len(X_valid)}, test:{len(X_test)}")

train = [[_y_train,_x_train] for _y_train,_x_train  in zip(y_train,X_train)]
valid = [[_y_valid,_x_valid] for _y_valid,_x_valid in zip(y_valid,X_valid)]
test = [[_y_test,_x_test] for _y_test,_x_test in zip(y_test,X_test)]

def save_data(data:List[List],file_name):
    with open(file_name, mode='w', encoding='utf-8') as f:
        writer = csv.writer(f,delimiter='\t')
        writer.writerows(data)

save_data(train,'files/train.txt')
save_data(valid,'files/valid.txt')
save_data(test,'files/test.txt')

y len :13340, X len: 13340
train:10672, valid:1334, test:1334


In [89]:
for i,(y,x) in enumerate(train):
    if '210714' in x:
        print(i,x)


In [91]:
!head files/train.txt

In [92]:
!grep 210714 files/train.txt

In [93]:
!cut -f 1 files/train.txt | sort | uniq -c

4538 b
4228 e
 701 m
1205 t


In [94]:
!cut -f 1 files/valid.txt | sort | uniq -c

 531 b
 529 e
 119 m
 155 t


In [95]:
!cut -f 1 files/test.txt | sort | uniq -c

 558 b
 522 e
  90 m
 164 t


# 51. 特徴量抽出
学習データ，検証データ，評価データから特徴量を抽出し，それぞれtrain.feature.txt，valid.feature.txt，test.feature.txtというファイル名で保存せよ． なお，カテゴリ分類に有用そうな特徴量は各自で自由に設計せよ．記事の見出しを単語列に変換したものが最低限のベースラインとなるであろう．

# 52. 学習
51で構築した学習データを用いて，ロジスティック回帰モデルを学習せよ．

# 53. 予測
52で学習したロジスティック回帰モデルを用い，与えられた記事見出しからカテゴリとその予測確率を計算するプログラムを実装せよ．

# 54. 正解率の計測
52で学習したロジスティック回帰モデルの正解率を，学習データおよび評価データ上で計測せよ．

# 55. 混同行列の作成
52で学習したロジスティック回帰モデルの混同行列（confusion matrix）を，学習データおよび評価データ上で作成せよ．

# 56. 適合率，再現率，F1スコアの計測
52で学習したロジスティック回帰モデルの適合率，再現率，F1スコアを，評価データ上で計測せよ．カテゴリごとに適合率，再現率，F1スコアを求め，カテゴリごとの性能をマイクロ平均（micro-average）とマクロ平均（macro-average）で統合せよ．

# 57. 特徴量の重みの確認
52で学習したロジスティック回帰モデルの中で，重みの高い特徴量トップ10と，重みの低い特徴量トップ10を確認せよ．

# 58. 正則化パラメータの変更
ロジスティック回帰モデルを学習するとき，正則化パラメータを調整することで，学習時の過学習（overfitting）の度合いを制御できる．異なる正則化パラメータでロジスティック回帰モデルを学習し，学習データ，検証データ，および評価データ上の正解率を求めよ．実験の結果は，正則化パラメータを横軸，正解率を縦軸としたグラフにまとめよ．



# 59. ハイパーパラメータの探索
学習アルゴリズムや学習パラメータを変えながら，カテゴリ分類モデルを学習せよ．検証データ上の正解率が最も高くなる学習アルゴリズム・パラメータを求めよ．また，その学習アルゴリズム・パラメータを用いたときの評価データ上の正解率を求めよ．

